In [ ]:
import requests, re
import torch
import pathlib
from transformers import GPTNeoXTokenizerFast

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from refuge._vendor.mkultra.tuning import GPTNeoXPromptTuningLM

In [ ]:
model_name = "databricks/dolly-v2-3b"

In [ ]:
from refuge._vendor.dolly.instruct_pipeline import InstructionTextGenerationPipeline

tokenizer = GPTNeoXTokenizerFast.from_pretrained(model_name, padding_side="left")
model = GPTNeoXPromptTuningLM.from_pretrained(model_name, device_map="auto")

In [ ]:
prompt = "Can you tell me about yourself and what you are aiming to achieve?"
model_inputs = tokenizer(prompt, return_tensors="pt")

input_ids = model_inputs["input_ids"]
attention_mask = model_inputs["attention_mask"]

generated_sequence_tensor = model.generate(
    input_ids=input_ids.to(model.device),
    attention_mask=attention_mask.to(model.device),
    pad_token_id=tokenizer.pad_token_id,
    max_new_tokens=128,
    top_p=0.92,
    do_sample=True
)

generated_sequence = generated_sequence_tensor.cpu().numpy().tolist()
print(tokenizer.decode(generated_sequence[0]))

In [ ]:
# MIT License

# Copyright (c) 2021 corolla-johnson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.


In [ ]:
def create_alice_txt(path):
    data_str = requests.get("https://www.gutenberg.org/files/11/11-0.txt").content.decode("utf-8")
    clean_data_str = data_str

    def regex_replace(str, regex, group, replacement):
        pat = re.compile(regex)
        while True:
            m = pat.search(str)
            if m is not None:
                str = str[:m.start(group)] + replacement + str[m.end(group):]
            else:
                break
        return str

    clean_data_str = regex_replace(clean_data_str, r"\r", 0, "")
    clean_data_str = regex_replace(clean_data_str, r"\S(\n)\S", 1, " ")
    clean_data_str = regex_replace(clean_data_str, r"\u201C", 0, '"')
    clean_data_str = regex_replace(clean_data_str, r"\u201D", 0, '"')
    clean_data_str = regex_replace(clean_data_str, r"_", 0, '')
    clean_data_str = clean_data_str[1434:-18595]

    with open(path, "w") as f:
        f.write(clean_data_str)
        
path = pathlib.Path("alice.txt")

if not path.exists():
    create_alice_txt(path)

In [ ]:
#-----------------------#
#  Training Parameters  #
#-----------------------#

# Use a string to set the initial value of the soft prompt.
# Be aware of the number of tokens.
initial_prompt = "A surreal children's fantasy story set in a subterranean world populated by peculiar anthropomorphic creatures.\n"

print(f"Initial prompt length: {len(tokenizer.encode(initial_prompt))} tokens")

# Decide the length of your training blocks in tokens.
# Safe sizes for gpt-neo-2.7B-halved:
#  - 700 on a Colab T4 (16GB)
#  - 400 on a Colab K80 (12GB)
#  - 32 on a GTX1080 (8GB)
# If it seems a bit small, don't worry!
# Soft prompts can be moved forward in context for the best effect.
block_size = 700

# Name your soft prompt project.
sp_name = 'alice-cyclic-dropout-2'

# What's the name of model you'll be using?
# e.g. gpt2, gpt2-large, gpt-neo-2.7B
# (This will be added to the project directory and soft prompt name)
# model_name = 'dolly-v2-3b'

# Specify the model directory or huggingface name.
# model_dir = 'databricks/dolly-v2-3b'
model_base_name = model_name.split("/")[-1]

# The above model_dir will download GPT2 1.5B from Huggingface as a baseline.
# It is recommended to use finetuneanon's FP16 fork of gpt-neo-2.7B, which can be downloaded from this magnet link:
# magnet:?xt=urn:btih:f50bb4e259d2f96aa9151443950b0d2b899a097c&dn=gpt-neo-2.7B-halved&tr=http%3A%2F%2Fopenbittorrent.com%3A80%2Fannounce&tr=http%3A%2F%2Ft.nyaatracker.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce
# Once you've saved it to your local machine, create a 'models' folder in your Google Drive and upload it there,
# then uncomment the following:
#model_dir = "/content/drive/MyDrive/models/gpt-neo-2.7B-halved/"

# Should be 'gpt2' or 'gpt-neo'.
model_type = 'gpt-neox'

# Specify the path to the text file used for training.
text_path = "alice.txt"
# You can also use something uploaded to your Google Drive, e.g.
#text_path = "/content/drive/MyDrive/datasets/nm_burning_chrome.txt"

# Specify the project directory.
project_dir = f"/home/simon/.mkultra/soft_prompts/{sp_name}-{model_base_name}/"

# Checkpoint interval in steps.
checkpoint_interval = 20

# Evaluation interval in steps.
eval_interval = 5

# How many blocks to use for evaluation.
eval_blocks = 16

# Adafactor hyperparameters
optimizer_params = {
    # Fixed learning rate, recommend 1e-4 to 1e-3
    "lr": 2e-4,
    
    # 1st momentum, recommend 0
    "beta1": 0.0,

    # 2nd momentum decay schedule, recommend -0.3 (lower is slower)
    "decay_rate": -0.8,

    # Weight decay, recommend 1e-5
    "weight_decay": 0.1,
    
    # Update scaling, recommend False
    "scale_parameter": False,
    
    # Built-in LR scheduler, recommend False
    "relative_step": False
    }

# LR scheduler parameters
scheduler_params = {
    "num_warmup_steps": 10,
    "num_cycles": 8,
    "num_training_steps": 1000
}

# (Use these for GPT-Neo)
#scheduler_params = {
#    "num_warmup_steps": 10,
#    "num_cycles": 4,
#    "num_training_steps": 240
#}

base_acc_steps = 16
acc_doubling_rate = 0
plateau_steps = 0

In [ ]:
#@title Initialize project
#@markdown This will load the latest checkpoint if the project directory already exists.

from refuge._vendor.mkultra.soft_prompt import SoftPrompt
from transformers import Adafactor
import os

filename_for_checkpoint = lambda step: f"{sp_name}-{model_base_name}-step-{step}.json"
loaded_sp = None
project_files = None

# Look for existing project directory
try:
    os.makedirs(project_dir)
    print(f"Created project directory at {project_dir}")
except FileExistsError:
    print(f"Found project directory at {project_dir}")

# Look for existing checkpoints
project_files = os.listdir(project_dir)
if project_files is not None:
    checkpoint_files = [check_file for check_file in project_files if ('-step-' in check_file) ]

    if len(checkpoint_files) > 0:
        highest_step = max([ int(check_file[check_file.rfind('-step-')+6:-5]) for check_file in checkpoint_files ])
        loaded_sp = SoftPrompt.from_file( os.path.join(project_dir, filename_for_checkpoint(highest_step)) )
        print(f"Loading latest checkpoint: {highest_step}")
    else:
        print("No checkpoints found")

In [ ]:
#@title Process dataset
#@markdown This will load an existing set
#@markdown of tokens if present in the project directory.

import json
import math

text_tokenized = None
tokens_path = os.path.join(project_dir,"tokens.json")

# See if we already have a tokens file
try:
    with open(tokens_path, 'r', encoding='utf-8') as file:
        text_tokenized = json.load(file)
        print("Loaded existing tokens.json file")

except FileNotFoundError:
    print("No tokens.json exists, creating it...")

# If not, make one now
if text_tokenized is None:

    with open(text_path, 'r', encoding='utf-8') as file:
        text = file.read()
    text_tokenized = tokenizer.encode(text)
    
    with open(tokens_path, 'x', encoding='utf-8') as file:
        json.dump(text_tokenized, file)

text_length = len(text_tokenized)
num_blocks = math.ceil(text_length/block_size)

print(f"Length of text: {len(text_tokenized)} tokens")
print(f"Number of blocks: {num_blocks}, each {block_size} tokens")

# Partition tokens into blocks
blocks = list()
for block_num in range(num_blocks):
    start = block_num * block_size
    end = min(start + block_size, text_length)
    blocks.append( text_tokenized[start:end] )

block_order_path = os.path.join(project_dir, "block_order.json")

# See if we already have a block_order file
try:
    with open(block_order_path, 'r', encoding='utf-8') as file:
        block_order = json.load(file)
        print("Loaded existing block_order.json file")

except FileNotFoundError:
    print("No block_order.json exists, creating it...")
    block_order = [*range(num_blocks)]

    with open(block_order_path, 'x', encoding='utf-8') as file:
        json.dump(block_order, file)

In [ ]:
#@title Initialize soft prompt in model
#@markdown If a checkpoint is present, use that.
if loaded_sp is None:
    initial_sp = SoftPrompt.from_string(initial_prompt, model, tokenizer)
    print(f"Initial prompt length: {len(initial_sp)}")
    model.set_soft_prompt(initial_sp)

    sp_step = 0
    eval_loss = 100
else:
    model.set_soft_prompt(loaded_sp)
    sp_step = loaded_sp._metadata['step']
    eval_loss = loaded_sp._metadata['loss']

In [ ]:
# Configure number of steps to train for.
# One step is (acc_steps) forward passes.
num_training_steps = scheduler_params['num_training_steps']

In [ ]:
from transformers import AdamW, Adafactor
import transformers

# Feed soft params to optimizer
optimizer_params['params'] = [model.get_soft_params()]
optimizer = Adafactor(**optimizer_params)
optimizer.state['step'] = sp_step

scheduler_params['optimizer'] = optimizer
scheduler = transformers.get_cosine_with_hard_restarts_schedule_with_warmup(**scheduler_params)

In [ ]:
#@title Train the soft prompt

from tqdm.notebook import tqdm
import random
import torch
import math

torch.cuda.empty_cache()
loss_log_path = os.path.join(project_dir,"loss_log.csv")
bar = tqdm(total=num_training_steps)
optimizer.state['step'] = sp_step
evals_since_last_improvement = 0
best_eval = float('inf')

# Fix eval order
eval_order = [*range(num_blocks)]
# random.seed(1234)
random.shuffle(eval_order)

# Function for gradient accumulation scheduling
def get_acc_steps(sp_step):
    if acc_doubling_rate != 0:
        return round(base_acc_steps * math.pow(2, (sp_step / acc_doubling_rate)))
    else:
        return base_acc_steps

for session_step in range(num_training_steps):
      model.train()

      acc_steps = get_acc_steps(sp_step)

      for i in range(acc_steps):
          idx = (sp_step*acc_steps + i) % num_blocks

          # Shuffle blocks every epoch
          if idx == 0:
              random.shuffle(block_order)
              with open(block_order_path, 'w', encoding='utf-8') as file:
                  json.dump(block_order, file)

          block = blocks[block_order[idx]]

          input_ids = torch.LongTensor(block).unsqueeze(0).cuda().detach()
          
          # Forward pass and optimize
          outputs = model(input_ids=input_ids, labels=input_ids)
          loss = outputs.loss
          loss.backward()

          instant_loss = loss.item()
          if math.isnan(instant_loss):
              torch.cuda.empty_cache()
              raise KeyboardInterrupt

          # Discard tensor that was moved to GPU
          del input_ids
          torch.cuda.empty_cache()

      # Accumulate gradients
      optimizer.step()
      lr = optimizer.param_groups[0]["lr"]
      scheduler.step()
      optimizer.zero_grad()

      if math.isnan(instant_loss):
          torch.cuda.empty_cache()
          raise KeyboardInterrupt

      # Evaluate model and plot loss
      if sp_step%eval_interval == 0:
          model.eval()
          torch.cuda.empty_cache()
          eval_loss = 0

          with torch.no_grad():
              for eval_step in range(eval_blocks):
                  block = blocks[eval_order[eval_step]]
                  input_ids = torch.LongTensor(block).unsqueeze(0).cuda().detach()
                  eval_loss += model(input_ids=input_ids, labels=input_ids).loss.item()
                  
                  # Discard tensor that was moved to GPU
                  del input_ids
                  torch.cuda.empty_cache()

          eval_loss /= eval_blocks

          with open(loss_log_path, 'a', encoding='utf-8') as file:
              file.write(f"{sp_step},{eval_loss}\n")
          
          # Stop if loss has plateaued
          if plateau_steps != 0:
              if eval_loss < best_eval:
                  best_eval = eval_loss
                  evals_since_last_improvement = 0
              else:
                  evals_since_last_improvement += 1
              if evals_since_last_improvement > plateau_steps:
                  print(f"No improvement for {plateau_steps} evals")
                  break

      # Save checkpoint every so often
      if sp_step%checkpoint_interval == 0:
          sp = SoftPrompt.from_tuning_model(model,
              {"name" : sp_name + f"-step-{sp_step}",
               "step"  : sp_step,
               "loss"  : eval_loss})
          sp.to_file( os.path.join( project_dir,filename_for_checkpoint(sp_step) ) )

      bar.set_postfix({
          "Model Step" : sp_step,
          "Eval Loss"  : "{el:.5f}".format(el=eval_loss),
          "Acc Steps"  : acc_steps,
          "LR"         : lr
      })
      bar.update(1)
      sp_step += 1

# Save a checkpoint once done
sp = SoftPrompt.from_tuning_model(model,
    {"name"  : sp_name + f"-step-{sp_step}",
     "step"  : sp_step,
     "loss"  : eval_loss})
sp.to_file( os.path.join( project_dir,filename_for_checkpoint(sp_step) ) )

In [ ]:
#@title Flush memory after interrupting training
#@markdown This will *hopefully* prevent a CUDA out-of-memory error.
try:
  del input_ids
except Exception:
  pass
torch.cuda.empty_cache()

In [ ]:
# Plot loss
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import numpy as np

fname2 = cbook.get_sample_data(loss_log_path, asfileobj=False)
with cbook.get_sample_data(loss_log_path) as file:
    array = np.loadtxt(file, delimiter=",")

fig = plt.figure()
plt.plot(array[:, 0], array[:, 1])

In [ ]:
# Try generating with your model
model.eval()

# Restore soft prompt from checkpoint
# (Use above graph to find a good stopping point and check project directory for valid checkpoints)
sp = SoftPrompt.from_file( os.path.join(project_dir, filename_for_checkpoint(400)) )
model.set_soft_prompt(sp)

test = "Alice sipped her tea as the white rabbit gloated about his vast collection of pocket watches"

call = tokenizer(test, return_tensors="pt").input_ids.cuda()

basic_output = model.generate(
    input_ids=call,
    do_sample=True,
    min_length=call.shape[-1] + 200,
    max_length=call.shape[-1] + 200,
    temperature=1.0,
    # tfs = 0.9,
    repetition_penalty = 3.0,
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(basic_output[0]))

In [ ]:
model.eval()

# Purge soft prompt for comparison.
model.initialize_soft_prompt(n_tokens=1)

test = "Alice sipped her tea as the white rabbit gloated about his vast collection of pocket watches"

call = tokenizer(test, return_tensors="pt").input_ids.cuda()

basic_output = model.generate(
    input_ids=call,
    do_sample=True,
    min_length=call.shape[-1] + 200,
    max_length=call.shape[-1] + 200,
    temperature=1.0,
    # tfs = 0.9,
    repetition_penalty = 3.0,
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(basic_output[0]))